In [1]:
using Pkg, CSV, DataFrames, Word2Vec, Languages, MLDataUtils, WordTokenizers, Random, LIBSVM, Statistics, JLD

In [2]:
path = "../../data/train.csv"
df = DataFrame(CSV.read(path, normalizenames = true))
head(df)

,_0,_1467810369,Mon_Apr_06_22_19_45_PDT_2009,NO_QUERY,_TheSpecialOne_,_switchfoot_http_twitpic_com_2y1zl_Awww_that_s_a_bummer_You_shoulda_got_David_Carr_of_Third_Day_to_do_it_D
,Int64,Int64,String,String,String,String
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug


### Preproccess data

In [3]:
rename!(df,:_0 => :"class_label")
rename!(df,:_1467810369 => :"id")
rename!(df,:Mon_Apr_06_22_19_45_PDT_2009 => :"date")
rename!(df,:NO_QUERY => :"flag")
rename!(df,:_TheSpecialOne_ => :"userid")
rename!(df,:_switchfoot_http_twitpic_com_2y1zl_Awww_that_s_a_bummer_You_shoulda_got_David_Carr_of_Third_Day_to_do_it_D => :"tweet")

,class_label,id,date,flag,userid
,Int64,Int64,String,String,String
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo


In [4]:
df.class_label = replace!(df.class_label, 0 => -1)
df.class_label = replace!(df.class_label, 4 => 1)

1599999-element Array{Int64,1}:
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
  ⋮
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1

In [5]:
df

,class_label,id,date,flag,userid
,Int64,Int64,String,String,String
1,-1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton
2,-1,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus
3,-1,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF
4,-1,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli
5,-1,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf
6,-1,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch
7,-1,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ
8,-1,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood
9,-1,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo


In [6]:
df.tweet

1599999-element Array{String,1}:
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 "@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds"
 "my whole body feels itchy and like its on fire "
 "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "
 "@Kwesidei not the whole crew "
 "Need a hug "
 "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?"
 "@Tatiana_K nope they didn't have it "
 "@twittera que me muera ? "
 "spring break in plain city... it's snowing "
 "I just re-pierced my ears "
 "@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . ."
 "@octolinz16 It it counts, idk why I did either. you never talk to me anymore "
 ⋮
 "@Roy_Everitt ha- good job. that's right - we gotta throw that #bigrun tag EVERYWHERE! I wanna get it trending be

In [7]:
names(df)

6-element Array{String,1}:
 "class_label"
 "id"
 "date"
 "flag"
 "userid"
 "tweet"

In [8]:
deletecols!(df, [:id, :date, :flag, :userid])

,class_label,tweet
,Int64,String
1,-1,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,-1,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,-1,my whole body feels itchy and like its on fire
4,-1,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,-1,@Kwesidei not the whole crew
6,-1,Need a hug
7,-1,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
8,-1,@Tatiana_K nope they didn't have it
9,-1,@twittera que me muera ?


In [9]:
colwise(x -> sum(ismissing.(x)), df)

2-element Array{Int64,1}:
 0
 0

In [10]:
decode_map = [0=>"Negative", 2=> "Neutral", 4=> "Positive"]
function decode_sentiment(label)
    return decode_map[int(label)]
end

decode_sentiment (generic function with 1 method)

In [11]:
count_2 = 0
count_0 = 0
count_4 = 0
combine(groupby(df, :class_label), nrow => :count)
count_0 = combine(groupby(df, :class_label), nrow => :count)[:"count"][:1]
count_4 = combine(groupby(df, :class_label), nrow => :count)[:"count"][:2]
totalNum = size(df)[:1]

1599999

### TWEET FORMATTING ###

In [15]:
# Variables
usernames = r"@\w+" # regex for usernames such as '@twitter_user123'
links = r"((https?:\/\/)|(www\.))\S+" # regex for links starting with 'http' or 'www.'
htmltags = r"&#?\w+;" # regex for html character entities such as '&lt;' or '&quot;'
repeatedletters = r"(.)\1{3,}" # regex for letters repeated three or more times, such as "hiii"
apostrophes = "'" # separate so contractions don't get spaced out e.g. 'can't' becomes 'cant' not 'can t'
special = [usernames, links, htmltags, apostrophes, repeatedletters]
stopword_list = replace.(stopwords(Languages.English), "'" => "")

488-element Array{String,1}:
 "a"
 "about"
 "above"
 "across"
 "after"
 "again"
 "against"
 "all"
 "almost"
 "alone"
 "along"
 "already"
 "also"
 ⋮
 "youd"
 "youll"
 "young"
 "younger"
 "youngest"
 "your"
 "youre"
 "yours"
 "yourself"
 "yourselves"
 "youve"
 "z"

In [16]:
remove(str, pat) = replace(str, pat => "")
replacewithspace(str, pat) = replace(str, pat => " ")
removespecial(str) = reduce(remove, special, init=str)
format(str) = lowercase(join(split(replace(str, r"[^A-Za-z ]" => " ")), " "))*" "
removenoise(tweet) = (format ∘ removespecial)(tweet)
removestopwords(str) = reduce(replacewithspace, stopword_list, init=str)

removestopwords (generic function with 1 method)

In [17]:
stopword_list = replace.(stopwords(Languages.English), "'" => "")

488-element Array{String,1}:
 "a"
 "about"
 "above"
 "across"
 "after"
 "again"
 "against"
 "all"
 "almost"
 "alone"
 "along"
 "already"
 "also"
 ⋮
 "youd"
 "youll"
 "young"
 "younger"
 "youngest"
 "your"
 "youre"
 "yours"
 "yourself"
 "yourselves"
 "youve"
 "z"

In [18]:
# Functions
remove(str, pat) = replace(str, pat => "")
removefromlist(list, pat) = filter(x->x!=pat, list)
removespecial(str) = reduce(remove, special, init=str)
format(str) = lowercase(join(split(replace(str, r"[^A-Za-z ]" => " ")), " "))*" "
removenoise(tweet) = (format ∘ removespecial)(tweet)
removestopwords(str) = join(reduce(removefromlist, stopword_list, init=split(str)), " ")


removestopwords (generic function with 1 method)

### Word2Vec Creation

In [20]:
# Functions
createcorpus(tweets) = removestopwords(join(removenoise.(tweets)))
function writecorpusfile()
    tweets = df.tweets
    corpus = createcorpus(tweets)
    open("tweets", "w") do io
        write(io, corpus)
    end
end
createword2vec() = word2vec("tweets", "tweets-vec.txt", verbose = true)

createword2vec (generic function with 1 method)

In [21]:
writecorpusfile()
createword2vec()

LoadError: ArgumentError: column name :tweets not found in the data frame; existing most similar names are: :tweet

In [22]:
x = []

Any[]

In [23]:
head(df)

,class_label,tweet
,Int64,String
1,-1,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,-1,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,-1,my whole body feels itchy and like its on fire
4,-1,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,-1,@Kwesidei not the whole crew
6,-1,Need a hug


In [24]:
denoise_df!(df) = df.tweet .= (removestopwords ∘ removenoise).(df.tweet)
denoise_df!(df)

1599999-element Array{String,1}:
 "upset update facebook texting cry result school blah"
 "dived times ball managed save rest bounds"
 "body feels itchy fire"
 "behaving mad"
 "crew"
 "hug"
 "hey time rains bit bit lol fine thanks"
 "nope"
 "que muera"
 "spring break plain city snowing"
 "re pierced ears"
 "bear watch ua loss embarrassing"
 "counts idk talk anymore"
 ⋮
 "ha job gotta throw bigrun tag wanna trending start"
 "glad ur"
 "xbox"
 "mmmm sounds absolutely perfect schedule time lay bed sunday ugh"
 "recovering weekend"
 ""
 "yeah waiting wonder time blog"
 "woke school feeling"
 "thewdb com cool hear walt interviews"
 "ready mojo makeover details"
 "happy th birthday boo alll time tupac amaru shakur"
 "happy charitytuesday"

In [26]:
head(df, 100)

,class_label,tweet
,Int64,String
1,-1,upset update facebook texting cry result school blah
2,-1,dived times ball managed save rest bounds
3,-1,body feels itchy fire
4,-1,behaving mad
5,-1,crew
6,-1,hug
7,-1,hey time rains bit bit lol fine thanks
8,-1,nope
9,-1,que muera


In [28]:
model = wordvectors("../../data/tweets-vec.txt")

WordVectors 71291 words, 100-element Float64 vectors

In [29]:
function createFeatureVector(sentence)
    wordCount = 0
    vector = zeros(size(model)[1])
    sentence_split = split(sentence)
    for word in sentence_split
        try
            vector =  vector + get_vector(model, word)
            wordCount = wordCount + 1
        catch
            nothing
        end
    end 
    if wordCount > 0
        vector = vector / wordCount
    end
    return vector
end

createFeatureVector (generic function with 1 method)

In [30]:
df = DataFrame(shuffle(eachrow(df)))

,class_label,tweet
,Int64,String
1,-1,lol poor feel shake start
2,-1,house sleep idk sister couch floor bit floor uncomforable
3,-1,atleast fake political drama parties votes tn name suffering eelam tamils game
4,-1,day argh
5,1,watching taylor swifts video blogs fun
6,-1,wow times update tx boring lol
7,-1,flawless win aiden win bet wil hav career dancin anyways coz amazin
8,-1,dyed hair red complete mess
9,1,watching movie text please


In [31]:
df[:2]

1599999-element Array{String,1}:
 "lol poor feel shake start"
 "house sleep idk sister couch floor bit floor uncomforable"
 "atleast fake political drama parties votes tn name suffering eelam tamils game"
 "day argh"
 "watching taylor swifts video blogs fun"
 "wow times update tx boring lol"
 "flawless win aiden win bet wil hav career dancin anyways coz amazin"
 "dyed hair red complete mess"
 "watching movie text please"
 "irish spring boxes hold single bar detect usual associated rant"
 "wats racism australia nice"
 "glad found helpful amount flexibility images amazing getting camera helps"
 "cooking police dept charity sumthing"
 ⋮
 "unfortunately"
 "cold"
 "night friends wine wish mike couldve"
 "pretty cool omg forgot preist song hats song movie mine"
 "damn alarm installation pda hanging"
 "night hours sleep logical homework wish couldve bamboozle yesterday"
 "woke late mail means netflix movie til monday boo"
 ""
 "lush day skating sweet weather thank god love peace"
 "day childm

In [32]:
features = hcat(createFeatureVector.(df[:2])...) # hcat converts Array{Array{Float64,1},1} into Array{Float64,2}
labels = df[:1]

1599999-element Array{Int64,1}:
 -1
 -1
 -1
 -1
  1
 -1
 -1
 -1
  1
  1
 -1
  1
  1
  ⋮
 -1
 -1
  1
  1
 -1
 -1
 -1
 -1
  1
 -1
  1
 -1

### Split data set into train and test 

In [35]:
x_train, x_test = splitobs(features, at = 0.8);
y_train, y_test = splitobs(labels, at = 0.8);

In [36]:
label_train = labels[1:10000]
label_test = labels[10001:20000]
feature_train = features[:,1:10000]
feature_test = features[:,10001:20000]

100×10000 Array{Float64,2}:
  0.0741487   -0.0621839   -0.0189069   …  -0.0332527   -0.0194043
 -0.0308018   -0.0693011   -0.0271978       0.00762508  -0.0635016
 -0.00811183  -0.0223302   -0.0652227      -0.0390625    0.054482
 -0.0383168    0.116287    -0.0706275      -0.0539178   -0.0366495
 -0.0561847    0.00782646  -0.010846        0.0163313   -0.0230247
  0.0091443    0.0693305    0.0251633   …   0.0762111   -0.045469
 -0.0111172    0.0130282   -0.0216771      -0.0066867   -0.041482
 -0.0665223   -0.0471712   -0.00806832      0.0220282    0.0139054
 -0.0265368    0.0950556   -0.0219129      -0.00623255  -0.0545038
  0.0389063   -0.0830055   -0.0425106       0.0425412    0.0506114
 -0.0289374   -0.013035    -0.0277565   …  -0.0386494   -0.0694394
  0.262685     0.120386     0.0713697       0.152552     0.161299
 -0.199201    -0.0282382   -0.061378       -0.0525073    0.00855805
  ⋮                                     ⋱               
 -0.0532545   -0.107636    -0.025055       -0.0

### Create SVM ML Model

In [37]:
model = svmtrain(feature_train, label_train)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 100, 2, [-1, 1], Int32[1, 2], Float64[], Int32[], LIBSVM.SupportVectors{Int64,Float64}(9736, Int32[4868, 4868], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-0.005917379082014472 0.0829597902807945 … 0.006929897485434017 -0.06046660864248539; -0.06335114994787047 -0.13393607660268106 … -0.11112761489968923 -0.07607712540547393; … ; 0.028997484216069215 -0.01114752897029144 … -0.07349718227927793 -0.03888879698450533; 0.012615253861914385 -0.023845778074134016 … 0.031893204476993474 0.012076859456112577], Int32[1, 2, 3, 4, 6, 7, 8, 11, 14, 20  …  9978, 9981, 9985, 9986, 9988, 9990, 9991, 9992, 9996, 9998], LIBSVM.SVMNode[LIBSVM.SVMNode(1, -0.005917379082014472), LIBSVM.SVMNode(1, 0.0829597902807945), LIBSVM.SVMNode(1, 0.012317784358104539), LIBSVM.SVMNode(1, -0.1575328271389042), LIBSVM.SVMNode(1, -0.04820014245282541), LIBSVM.SVMNode(1, -0.007869881598971269), LIBSVM.SVMNode(1, 0.11945052021111402

In [38]:
@save "model2.jld" model

In [40]:
(predicted_labels, decision_values) = svmpredict(model,feature_test)

([-1, -1, 1, 1, -1, 1, -1, 1, -1, -1  …  -1, 1, 1, -1, -1, -1, -1, 1, 1, -1], [0.17947528906498433 0.2647499843566993 … -0.3920475110428713 0.074172762668819; 0.0 0.0 … 0.0 0.0])

In [41]:
print("Accuracy: ")
println(mean((predicted_labels .== label_test))*100)

Accuracy: 61.27


In [42]:
model

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 100, 2, [-1, 1], Int32[1, 2], Float64[], Int32[], LIBSVM.SupportVectors{Int64,Float64}(9736, Int32[4868, 4868], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-0.005917379082014472 0.0829597902807945 … 0.006929897485434017 -0.06046660864248539; -0.06335114994787047 -0.13393607660268106 … -0.11112761489968923 -0.07607712540547393; … ; 0.028997484216069215 -0.01114752897029144 … -0.07349718227927793 -0.03888879698450533; 0.012615253861914385 -0.023845778074134016 … 0.031893204476993474 0.012076859456112577], Int32[1, 2, 3, 4, 6, 7, 8, 11, 14, 20  …  9978, 9981, 9985, 9986, 9988, 9990, 9991, 9992, 9996, 9998], LIBSVM.SVMNode[LIBSVM.SVMNode(1, -0.005917379082014472), LIBSVM.SVMNode(1, 0.0829597902807945), LIBSVM.SVMNode(1, 0.012317784358104539), LIBSVM.SVMNode(1, -0.1575328271389042), LIBSVM.SVMNode(1, -0.04820014245282541), LIBSVM.SVMNode(1, -0.007869881598971269), LIBSVM.SVMNode(1, 0.11945052021111402

In [44]:
using BSON: @save
@save "../../data/mymodel2.bson" model